In [ ]:
import pandas as pd

In [2]:
file_path= "../data/"
list_col = ['HES US Equity', 'NEM US Equity', 'INTC US Equity', 'IFF US Equity',
       'MOS US Equity', 'EXC US Equity', 'KMB US Equity', 'JNJ US Equity',
       'BAX US Equity', 'F US Equity', 'CC US Equity', 'COP US Equity',
       'WFC US Equity', 'WY US Equity', 'TGT US Equity', 'MMM US Equity',
       'GM US Equity', 'CMI US Equity', 'HAS US Equity', 'DUK US Equity',
       'EMN US Equity', 'BK US Equity', 'UFS US Equity', 'ECL US Equity',
       'SLB US Equity', 'UPS US Equity', 'T US Equity', 'NSC US Equity',
       'PPL US Equity', 'MO US Equity', 'JLL US Equity', 'C US Equity',
       'ABT US Equity', 'AMD US Equity', 'CVX US Equity', 'CMA US Equity',
       'DTE US Equity', 'HSY US Equity', 'KIM US Equity', 'NBL US Equity',
       'IBM US Equity', 'WELL US Equity', 'IR US Equity', 'BKR US Equity',
       'WEC US Equity', 'OXY US Equity', 'WMB US Equity', 'UNP US Equity',
       'WM US Equity', 'CCL US Equity']

In [3]:
returns = pd.read_csv(file_path+"return.csv")[['Date1']+list_col]
returns.rename(columns={returns.columns[0]: "Date" }, inplace = True)
CURRENT_EV_TO_T12M_EBITDA = pd.read_csv(file_path+"CURRENT_EV_TO_T12M_EBITDA.csv")[['Date']+list_col]
DIVIDEND_INDICATED_YIELD= pd.read_csv(file_path+"DIVIDEND_INDICATED_YIELD.csv")[['Date']+list_col]
EBITDA_TO_INTEREST_EXPN = pd.read_csv(file_path+"EBITDA_TO_INTEREST_EXPN.csv")[['Date']+list_col]
EBITDA_TO_REVENUE = pd.read_csv(file_path+"EBITDA_TO_REVENUE.csv")[['Date']+list_col]
EPS_GROWTH = pd.read_csv(file_path+"EPS_GROWTH.csv")[['Date']+list_col]
EQY_DVD_YLD_IND = pd.read_csv(file_path+"EQY_DVD_YLD_IND.csv")[['Date']+list_col]
PE_RATIO = pd.read_csv(file_path+"PE_RATIO.csv")[['Date']+list_col]
TOT_DEBT_TO_EBITDA = pd.read_csv(file_path+"TOT_DEBT_TO_EBITDA.csv")[['Date']+list_col]

In [4]:
list_df_str = [
            'CURRENT_EV_TO_T12M_EBITDA','DIVIDEND_INDICATED_YIELD','EBITDA_TO_INTEREST_EXPN',
            'EBITDA_TO_REVENUE','EPS_GROWTH','EQY_DVD_YLD_IND','PE_RATIO','TOT_DEBT_TO_EBITDA'
          ]
list_df = [
            CURRENT_EV_TO_T12M_EBITDA,DIVIDEND_INDICATED_YIELD,EBITDA_TO_INTEREST_EXPN,
            EBITDA_TO_REVENUE,EPS_GROWTH,EQY_DVD_YLD_IND,PE_RATIO,TOT_DEBT_TO_EBITDA
          ]
len(list_df)

8

In [5]:
col = 'HES US Equity'
df = returns[["Date",str(col)]].copy()
df.Date = pd.to_datetime(df.Date)
df.Date = df.Date.dt.strftime('%m/%Y')
df_final = df.copy()

for res,res_str in zip(list_df,list_df_str):
    
    df2 = res[["Date",str(col)]].copy()
    df2.Date = pd.to_datetime(df2.Date)
    df2.rename(columns={str(col): str(res_str) }, inplace = True)
    df2.Date = df2.Date + pd.DateOffset(months=1)
    df2.Date = df2.Date.dt.strftime('%m/%Y')
    df_final = pd.merge(df_final,df2,on='Date', how='outer').fillna(method='ffill')

df_final.Date = pd.to_datetime(df_final.Date)
print(df_final.shape)
df_final.head(20)

(119, 10)


,Date,HES US Equity,CURRENT_EV_TO_T12M_EBITDA,DIVIDEND_INDICATED_YIELD,EBITDA_TO_INTEREST_EXPN,EBITDA_TO_REVENUE,EPS_GROWTH,EQY_DVD_YLD_IND,PE_RATIO,TOT_DEBT_TO_EBITDA
0,2009-12-01,-4.479339,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-01-01,1.747707,5.5099,0.6612,16.5495,17.3542,NaN,0.6612,40.8164,1.0684
2,2010-02-01,6.552083,5.5099,0.6612,16.5495,17.3542,NaN,0.6612,40.8164,1.0684
3,2010-03-01,1.598721,5.5099,0.6612,16.5495,17.3542,NaN,0.6612,40.8164,1.0684
4,2010-04-01,-16.286390,4.5871,0.6395,19.4167,17.6153,NaN,0.6395,18.0571,0.8424
5,2010-05-01,-5.199633,4.5871,0.6395,19.4167,17.6153,NaN,0.6395,18.0571,0.8424
6,2010-06-01,6.456099,4.5871,0.6395,19.4167,17.6153,NaN,0.6395,18.0571,0.8424
7,2010-07-01,-6.232506,3.5101,0.7946,15.9036,17.0719,270.9677,0.7946,10.9660,0.7725
8,2010-08-01,17.862510,3.5101,0.7946,15.9036,17.0719,270.9677,0.7946,10.9660,0.7725
9,2010-09-01,6.613667,3.5101,0.7946,15.9036,17.0719,270.9677,0.7946,10.9660,0.7725


In [ ]:
# Creation d'une liste avec les modeles qui vont être appliqués
models = []
models.append(LogisticRegression())
models.append(RandomForestClassifier())
models.append(XGBClassifier())
models.append(DecisionTreeClassifier())
models.append(LinearDiscriminantAnalysis())

In [ ]:
i = 0 
results = []
date_of_pred = []


In [ ]:
# def random_search(model):
#     typeofmodel = str(model).split('(')[0]
#     print(typeofmodel)
    
#     if typeofmodel == "XGBRegressor":
#         param_grid = {
#             "learning_rate" : [0.05,0.01]
#         }
    
#     search = RandomizedSearchCV(estimator = model, param_distributions = param_grid)
#     search.fit(X_tuning,y_tuning)
#     y_pred = search.predict(X_test)
#     sc = rmse(y_test, y_pred)
    
#     return sc

In [ ]:
# Creation des listes vides qui vont contenir les scores
results = []
names = []
accur = []
precis = []
rec = []

# Recuperation du nom des modeles
for m in models:
    m = str(m).split('(')[0]
    names.append(m)
    
# Application de modeles
"""
Le but ici etait d'appliquer plusieurs modèles et de voir lequel performe le mieux avec notre base de données.
Les modeles choisis vont être comparés sur plusieures critères et la boucle en bas va ressortir pour chaque modele
la matrice de confusions et la courbe ROC avec le score de l'AUC.

Au final, un DataFrame sera fourni avec les résultats des différents modèles contenant :
 - AUC
 - Accuracy
 - Precision 
 - Recall
"""
for model in models:

    mod  = model
    mod.fit(X_train,y_train)
    y_pred=mod.predict(X_test)
    
    accur.append(metrics.accuracy_score(y_test, y_pred))
    precis.append(metrics.precision_score(y_test, y_pred))
    rec.append(metrics.recall_score(y_test, y_pred))
    cnf_matrix = confusion_matrix(y_test, y_pred)
    
    sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
#     ax.xaxis.set_label_position("top")
    plt.tight_layout()
    plt.title('Confusion matrix '+str(str(model).split('(')[0]), y=1.1)
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')
    plt.show()
    
    y_pred_proba = mod.predict_proba(X_test)[::,1]
    fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
    auc = metrics.roc_auc_score(y_test, y_pred_proba)
    plt.plot(fpr,tpr,label="AUC = "+str(round(auc,2)))
    plt.title(str(str(model).split('(')[0]))
    plt.legend(loc=4)
    plt.show()
    
    
    results.append(auc)

In [ ]:
results_model = pd.DataFrame(([results,accur,precis,rec]),columns = names) 
results_model.set_index([pd.Index(['AUC', 'Accuracy', 'Precision', 'Recall'])],inplace = True)
results_model.T